In [1]:
import time

import numpy as np
np.random.seed(1234)

from functools import reduce

import scipy.io
from scipy.interpolate import griddata
from sklearn.preprocessing import scale
# from utils import augment_EEG, cart2sph, pol2cart

######### import DNN for training using GPUs #########
from keras.utils.training_utils import multi_gpu_model

######### import DNN frameworks #########
import tensorflow as tf
import keras

# import high level optimizers, models and layers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import InputLayer

# for CNN
from keras.layers import Conv2D, MaxPooling2D
# for RNN
from keras.layers import LSTM

# for different layer functionality
from keras.layers import Dense, Dropout, Flatten

# utility functionality for keras
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding

# from keras import backend as K

Using TensorFlow backend.


# 1. Import in Data Necessary
Here, we can import the MNIST, or IMDB dataset for proof-of-concept. We also provide code for importing iEEG recording data, and how to transform them into input that can be provided to the DNN models built in section 2.

In [ ]:
from keras.datasets import imdb
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print len(mnist)
print type(mnist)

In [ ]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [2]:
# import raw data

# perform signal processing - FFT



In [3]:
# save data


In [4]:
# load back in data and augment dataset


# 2. Preprocess Data
Here, we preprocess data by producing the final set of images needed to input into the DNN model.

We first augment the dataset by applying transformations that the model will be invariant to (e.g. rotation, translation, etc.). 

Then we will mesh the data to fill in any missing data.

# 3. Build DNN Model
Here, we build the DNN model that will need to be trained. It will consist of a CNN-RNN model that has a VGG style CNN model with LSTM used for the RNN.

These will be capable of efficiently learning spatial patterns in the heatmaps fed in, and also capable of learning complex timing behavior from the recurrent neural network.

In [7]:
from ieeg_cnn_rnn import IEEGdnn

In [8]:
imsize=32  # the imsize dimension
n_colors=4 # the number of frequency bands we use can correpond

###### CNN Parameters #######
n_layers = (4,2,1)  # the number of layers of convolution
poolsize=(2,2)      # the size of the pooling done in 2D
n_outunits = 2      # the size of the output of the model (# classes)
n_fcunits = 1024    # the size of the fully connected layer at output

##### Optimizer Parameters #######
loss='categorical_crossentropy'
ADAM = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
metrics = ['accuracy']

# initialize the ieeg dnn model
ieegdnn = IEEGdnn(imsize, n_colors)
ieegdnn.build_cnn(w_init=None, n_layers=n_layers,poolsize=(2,2))
ieegdnn.build_output(n_outunits=n_outunits, n_fcunits=n_fcunits)

print ieegdnn.model.output

# ieegdnn.compile_model(loss=loss, optimizer=ADAM, metrics=metrics)

Tensor("dense_2/Softmax:0", shape=(?, 2), dtype=float32)


In [10]:
display(ieegdnn.model_config)

[{'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 32, 32, 4),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1),
   'dtype': 'float32',
   'filters': 32,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'kernel_size': (3, 3),
   'name': 'conv2d_1',
   'padding': 'valid',
   'strides': (1, 1),
   'trainable': True,
   'use_bias': True}},
 {'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 32, 32, 4),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': '

In [ ]:
####### RNN Parameters ######
num_units = 128
grad_clipping = 110
nonlinearity = keras.activations.tanh

ieegdnn.build_rnn(num_units=num_units, grad_clipping=grad_clipping, nonlinearity=nonlinearity)



# 4. Train Model and Test
Here, we run the training on gpu(s) and document the entire training time, and visualize the output produced.